day : Day of the Week
quarter : A portion of the month. A month was divided into four quarters
department : Associated department with the instance
team_no : Associated team number with the instance
no_of_workers : Number of workers in each team
no_of_style_change : Number of changes in the style of a particular product
targeted_productivity : Targeted productivity set by the Authority for each team for each day.
smv : Standard Minute Value, it is the allocated time for a task
wip : Work in progress. Includes the number of unfinished items for products
over_time : Represents the amount of overtime by each team in minutes
incentive : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
idle_time : The amount of time when the production was interrupted due to several reasons
idle_men : The number of workers who were idle due to production interruption
actual_productivity : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor,HistGradientBoostingRegressor
from sklearn.impute import SimpleImputer

In [2]:
train_data = pd.read_csv('train_dataset.csv')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017 entries, 0 to 1016
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   team                   1017 non-null   int64  
 1   targeted_productivity  1017 non-null   float64
 2   smv                    1017 non-null   float64
 3   wip                    594 non-null    float64
 4   over_time              1017 non-null   int64  
 5   incentive              1017 non-null   int64  
 6   idle_time              1017 non-null   float64
 7   idle_men               1017 non-null   int64  
 8   no_of_style_change     1017 non-null   int64  
 9   no_of_workers          1017 non-null   float64
 10  month                  1017 non-null   int64  
 11  quarter_Quarter1       1017 non-null   int64  
 12  quarter_Quarter2       1017 non-null   int64  
 13  quarter_Quarter3       1017 non-null   int64  
 14  quarter_Quarter4       1017 non-null   int64  
 15  quar

In [3]:
train_data.isna().sum()

team                       0
targeted_productivity      0
smv                        0
wip                      423
over_time                  0
incentive                  0
idle_time                  0
idle_men                   0
no_of_style_change         0
no_of_workers              0
month                      0
quarter_Quarter1           0
quarter_Quarter2           0
quarter_Quarter3           0
quarter_Quarter4           0
quarter_Quarter5           0
department_finishing       0
department_finishing       0
department_sweing          0
day_Monday                 0
day_Saturday               0
day_Sunday                 0
day_Thursday               0
day_Tuesday                0
day_Wednesday              0
actual_productivity        0
dtype: int64

In [4]:
train_data['wip']=train_data['wip'].fillna(train_data['wip'].mean())

In [5]:
train_data.isna().sum()

team                     0
targeted_productivity    0
smv                      0
wip                      0
over_time                0
incentive                0
idle_time                0
idle_men                 0
no_of_style_change       0
no_of_workers            0
month                    0
quarter_Quarter1         0
quarter_Quarter2         0
quarter_Quarter3         0
quarter_Quarter4         0
quarter_Quarter5         0
department_finishing     0
department_finishing     0
department_sweing        0
day_Monday               0
day_Saturday             0
day_Sunday               0
day_Thursday             0
day_Tuesday              0
day_Wednesday            0
actual_productivity      0
dtype: int64

In [6]:
corr = train_data.corr()

print(corr['actual_productivity'].reset_index().sort_values(by='actual_productivity',ascending=False))


                    index  actual_productivity
25    actual_productivity             1.000000
1   targeted_productivity             0.401069
17  department_finishing              0.143885
15       quarter_Quarter5             0.119284
3                     wip             0.087362
11       quarter_Quarter1             0.082608
5               incentive             0.074926
20           day_Saturday             0.045212
23            day_Tuesday             0.035301
12       quarter_Quarter2             0.020185
19             day_Monday            -0.004186
24          day_Wednesday            -0.020523
21             day_Sunday            -0.024243
22           day_Thursday            -0.029730
4               over_time            -0.033712
16   department_finishing            -0.045173
9           no_of_workers            -0.053273
6               idle_time            -0.058365
18      department_sweing            -0.083030
13       quarter_Quarter3            -0.086748
14       quar

In [7]:
X= train_data.iloc[:,[0,1,2,3,7,8,10,15,17]] 
y= train_data['actual_productivity']

In [8]:
X_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [9]:
# # Create an instance of SimpleImputer to impute missing values
# imputer = SimpleImputer(strategy='mean')

# # Fit the imputer to the training data and transform the training data
# X_train_imputed = imputer.fit_transform(X_train)

# # Transform the test data using the trained imputer
# X_test_imputed = imputer.transform(x_test)
# X_train_imputed.isna().sum()

In [29]:
# Define the parameter grid for GridSearchCV


# Create an instance of the Random Forest regressor
rf = RandomForestRegressor(random_state=42)

# Create an instance of GridSearchCV

# Fit the GridSearchCV object to the training data
rf.fit(X_train, y_train)
param_grid = {
    'n_estimators': [100, 200, 300],
    'min_samples_split': [2,3,4,5],
    'min_samples_leaf': [1, 2, 3],
    'criterion': ['squared_error', 'absolute_error']}

# Create an instance of GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error')

# Fit the grid search to the imputed training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print('Best Parameters:', best_params)
print('Best Score:', best_score)



Best Parameters: {'criterion': 'squared_error', 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 100}
Best Score: -0.017977701397973965


In [11]:
# test_data = pd.read_csv('test_dataset.csv')
# x_test= train_data.iloc[:,[0,1,2,3,7,8,10,15,17]] 
# y_test = test_data['actual_productivity']

In [30]:
from sklearn.metrics import r2_score
y_pred = grid_search.predict(x_test)
r2 = r2_score(y_test,y_pred)
print(r2*100)

31.11294231955526


In [31]:
from joblib import dump,load
dump(grid_search,'rf_grid.joblib')

['rf_grid.joblib']

In [32]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 711 entries, 810 to 102
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   team                   711 non-null    int64  
 1   targeted_productivity  711 non-null    float64
 2   smv                    711 non-null    float64
 3   wip                    711 non-null    float64
 4   idle_men               711 non-null    int64  
 5   no_of_style_change     711 non-null    int64  
 6   month                  711 non-null    int64  
 7   quarter_Quarter5       711 non-null    int64  
 8   department_finishing   711 non-null    int64  
dtypes: float64(3), int64(6)
memory usage: 55.5 KB


In [39]:
from sklearn.ensemble import HistGradientBoostingRegressor
Hgr = HistGradientBoostingRegressor()

Hgr.fit(X_train,y_train)

HistGradientBoostingRegressor()

In [49]:
Hgr_y_pred = Hgr.predict(X_train)
R2HGr=r2_score(y_train, Hgr_y_pred)
print('R2 for HistGradientBoostingRegressor	: ', R2HGr)

R2 for HistGradientBoostingRegressor	:  0.6724136851313154


In [50]:
dump(Hgr,'Hgr_model.joblib')

['Hgr_model.joblib']

In [ ]:
import gradio as gr
import pandas as pd
import joblib

# Load the pre-trained model
model = joblib.load('Hgr_model.joblib')

# Define the prediction function
def predict_performance(team, targeted_productivity, smv, wip, idle_men, no_of_style_change, month, quarter_Quarter5, department_finishing):
    input_data = pd.DataFrame([[team, targeted_productivity, smv, wip, idle_men, no_of_style_change, month, quarter_Quarter5, department_finishing]], columns=selected_features)
    prediction = model.predict(input_data)[0]
    return prediction*100

# Define the selected features in the specified sequence
selected_features = ['Team No.', 'Targeted Productivity(0 to 1)', 'Standard Minute Value(in min)', 'Tasks in progress(count)', 'Idle men(0/1)', 'No. of Style Change(count)', 'month', 'Is Quarter 5 ?(1/0)', 'Is This final stage Department?(0/1)']

# Create the input and output interfaces
inputs = [gr.inputs.Number(label=feature_name) for feature_name in selected_features]
output = gr.outputs.Textbox(label='Actual Productivity')

# Create the Gradio interface
interface = gr.Interface(fn=predict_performance, inputs=inputs, outputs=output, title='Employee Performance Prediction')

# Run the interface
interface.launch(share=True)
